In [1]:
import pandas as pd
from datetime import datetime
from nba_api.stats.endpoints import playergamelog
from sqlalchemy.dialects.postgresql import insert
from models import Game
import date_utils as date_mng
from data_manager import DataManager

data_manager = DataManager()
self = data_manager
print(self.db_player_id_map)
print(self.nba_player_id_map)
# self.sync_all_team_and_player_records()
self.sync_games(season="2023-24", season_type="Regular Season")

{201950: 92, 202083: 17, 1630180: 18, 1628401: 95, 1630552: 1, 1629052: 97, 1641754: 3, 1641723: 4, 1630214: 100, 1629631: 7, 1629640: 1953, 1629660: 177, 1627747: 178, 1628369: 91, 1627777: 186, 1627759: 93, 204001: 94, 1630600: 185, 1630202: 96, 1631100: 9, 203991: 10, 1630531: 105, 1631243: 12, 1629685: 13, 1628981: 14, 1629726: 15, 203992: 8, 1630596: 179, 1628370: 1861, 1631096: 2045, 1641809: 98, 1631120: 99, 1629622: 176, 1641775: 101, 1630573: 102, 1628436: 103, 201143: 104, 1631250: 181, 1629004: 106, 1629674: 107, 1630241: 180, 1641854: 182, 1629636: 183, 202684: 184, 1631230: 1963, 1641734: 187, 202694: 188, 1630556: 1871, 1629027: 6, 1630249: 16, 1630168: 11, 1630235: 2, 1627936: 356, 203468: 268, 1630529: 269, 1630245: 360, 1630700: 271, 1631207: 366, 1627742: 272, 1628386: 190, 1629731: 191, 1630171: 192, 1628378: 193, 1628984: 1954, 203897: 357, 1630172: 367, 1630929: 368, 1630631: 273, 202685: 274, 1629750: 365, 1641722: 276, 202696: 358, 201942: 359, 1630702: 443, 1628

In [ ]:
import pandas as pd
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from db_config import get_database_engine
from models import Team, Player, TradTeamStats, AdvTeamStats, TradPlayerStats, AdvPlayerStats, Game

# Initialize database connection
engine = get_database_engine()
Session = sessionmaker(bind=engine)
session = Session()

# Fetch teams with specified nicknames
team_nicknames = ["Nuggets", "Timberwolves"]
teams = session.query(Team).filter(Team.nickname.in_(team_nicknames)).all()

# Initialize a new Excel workbook
wb = Workbook()

# Function to add a DataFrame to an Excel sheet
def add_df_to_sheet(df, sheet_name):
    ws = wb.create_sheet(title=sheet_name)
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

# Function to convert SQLAlchemy objects to DataFrame
def convert_to_df(query_result, exclude_columns=['_sa_instance_state']):
    if not query_result:
        return pd.DataFrame()
    data = [{key: value for key, value in vars(obj).items() if key not in exclude_columns} for obj in query_result]
    return pd.DataFrame(data)

# Fetch and process team stats
for team in teams:
    trad_team_stats = session.query(TradTeamStats).filter(TradTeamStats.team_id == team.id).all()
    adv_team_stats = session.query(AdvTeamStats).filter(AdvTeamStats.team_id == team.id).all()
    
    # Convert to DataFrames
    trad_team_stats_df = convert_to_df(trad_team_stats)
    adv_team_stats_df = convert_to_df(adv_team_stats)
    
    # Debug prints to check DataFrame contents
    print(f"Team: {team.nickname} - Traditional Team Stats DataFrame Columns: {trad_team_stats_df.columns}")
    print(f"Team: {team.nickname} - Advanced Team Stats DataFrame Columns: {adv_team_stats_df.columns}")
    
    if not trad_team_stats_df.empty:
        # Add team and date columns
        trad_team_stats_df['team_name'] = team.full_name
        trad_team_stats_df['date'] = trad_team_stats_df['game_id'].apply(lambda x: session.query(Game).filter(Game.id == x).one().date)
        trad_team_stats_df = trad_team_stats_df.sort_values(by='date', ascending=False)
        add_df_to_sheet(trad_team_stats_df, f"{team.nickname}_trad_stats")

    if not adv_team_stats_df.empty:
        # Add team and date columns
        adv_team_stats_df['team_name'] = team.full_name
        adv_team_stats_df['date'] = adv_team_stats_df['game_id'].apply(lambda x: session.query(Game).filter(Game.id == x).one().date)
        adv_team_stats_df = adv_team_stats_df.sort_values(by='date', ascending=False)
        add_df_to_sheet(adv_team_stats_df, f"{team.nickname}_adv_stats")

# Fetch and process player stats
for team in teams:
    players = session.query(Player).filter(Player.team_id == team.id).all()
    for player in players:
        trad_player_stats = session.query(TradPlayerStats).filter(TradPlayerStats.player_id == player.id).all()
        adv_player_stats = session.query(AdvPlayerStats).filter(AdvPlayerStats.player_id == player.id).all()
        
        # Convert to DataFrames
        trad_player_stats_df = convert_to_df(trad_player_stats)
        adv_player_stats_df = convert_to_df(adv_player_stats)
        
        # Debug prints to check DataFrame contents
        print(f"Player: {player.name} - Traditional Player Stats DataFrame Columns: {trad_player_stats_df.columns}")
        print(f"Player: {player.name} - Advanced Player Stats DataFrame Columns: {adv_player_stats_df.columns}")
        
        if not trad_player_stats_df.empty:
            # Add player and date columns
            trad_player_stats_df['player_name'] = player.name
            trad_player_stats_df['date'] = trad_player_stats_df['game_id'].apply(lambda x: session.query(Game).filter(Game.id == x).one().date)
            trad_player_stats_df = trad_player_stats_df.sort_values(by='date', ascending=False)
            add_df_to_sheet(trad_player_stats_df, f"{player.nickname}_trad_stats")

        if not adv_player_stats_df.empty:
            # Add player and date columns
            adv_player_stats_df['player_name'] = player.name
            adv_player_stats_df['date'] = adv_player_stats_df['game_id'].apply(lambda x: session.query(Game).filter(Game.id == x).one().date)
            adv_player_stats_df = adv_player_stats_df.sort_values(by='date', ascending=False)
            add_df_to_sheet(adv_player_stats_df, f"{player.nickname}_adv_stats")

# Remove the default sheet created by openpyxl
wb.remove(wb['Sheet'])

# Save the workbook
wb.save('team_player_stats.xlsx')

# Close the session
session.close()
